In [1]:
%env TOKENIZERS_PARALLELISM=true
! pip install sentence-transformers

env: TOKENIZERS_PARALLELISM=true
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=7d21e9d00e684743558f17d2137529a988d164c282ae7749a7e8e37ae56aa768
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd
stop_words_df = pd.read_csv(
    header=None,
    filepath_or_buffer='https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK\'s%2520list%2520of%2520english%2520stopwords')
stopwords = stop_words_df[0].tolist() + []

In [3]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/shoppersentiments/TeePublic_review.csv', encoding='ISO-8859-1')
df.head()

,reviewer_id,store_location,latitude,longitude,date,month,year,title,review,review-label
0,0.0,US,37.090240,-95.712891,2023,6,2015 00:00:00,Great help with lost order,I had an order that was lost in transit. When ...,5
1,1.0,US,37.090240,-95.712891,2023,6,2024 00:00:00,I ordered the wrong size tee and hadï¿½ï¿½ï¿½,I ordered the wrong size tee and had difficult...,5
2,2.0,US,37.090240,-95.712891,2023,6,2017 00:00:00,These guys offer the best customerï¿½ï¿½ï¿½,These guys offer the best customer service in ...,5
3,3.0,US,37.090240,-95.712891,2023,6,2024 00:00:00,Good Stuff,Looked for an obscure phrase on a shirt. Teepu...,5
4,4.0,CA,56.130366,-106.346771,2023,6,2023 00:00:00,My order arrived in a good timelyï¿½ï¿½ï¿½,My order arrived in a good timely fashion & th...,4


In [4]:
chars = ['ï', '¿', '½', 'ý', '_', '~', '='] 
def clean(arg: str) -> str:
    result = arg
    for char in chars:
        result = result.replace(char, ' ')
    result = ' '.join(result.split())
    return result

SAMPLE_SIZE = 23 * 23
# we only want the text, and we want a small sample for performance reasons
review_df = df[['review', 'review-label']].copy().drop_duplicates(ignore_index=True).dropna().sample(n=SAMPLE_SIZE, random_state=2023)
review_df['token count'] = review_df['review'].apply(func=lambda x: len(x.split()))

sentences = review_df['review'].values.tolist()
# we need to remove trash tokens because they are fooling the transformer
sentences = [clean(item) for item in sentences]

review_df.shape

(529, 3)

In [5]:
from collections import Counter
character_keep = set(list('1234567890qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM'))
character_count = Counter(list(' '.join(sentences)))
characters = [item for item in dict(character_count).keys() if item not in character_keep]
print(characters)

[' ', '.', "'", '%', '!', ',', '-', '"', '?', '/', '$', '+', '&', ';', '(', ')', ':', '<']


In [6]:
from plotly.express import pie
pie(data_frame=review_df['review-label'].value_counts().to_frame().reset_index(), names='review-label', values='count')

Three quarters of our reviews are positive.

In [7]:
from plotly.express import histogram
histogram(data_frame=review_df, x='token count', log_y=True)

In [8]:
from arrow import now
from sentence_transformers import SentenceTransformer
from numpy.linalg import norm

embedding_start = now()
model_name = 'distilbert-base-nli-mean-tokens'
model = SentenceTransformer(model_name_or_path=model_name)
model.max_seq_length = 256
print('{}: max sequence length: {}'.format(now().time(), model.max_seq_length))
print('{}: loaded model: {}'.format(now().time(), model_name))

embedding = model.encode(sentences=sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
# normalize the embeddings
embedding = embedding / norm(embedding, axis=1, keepdims=True)

print('{}: got all embeddings'.format(now().time()))
print('embedding time: {}'.format(now() - embedding_start))

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

20:33:39.482393: max sequence length: 256
20:33:39.482666: loaded model: distilbert-base-nli-mean-tokens


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

20:34:06.561733: got all embeddings
embedding time: 0:00:38.525089


In [9]:
from arrow import now
from sentence_transformers.util import community_detection
from sklearn.cluster import AgglomerativeClustering

cluster_start = now()
clustering_choices = ['agglomerative', 'community']
clustering = clustering_choices[1]

if clustering == clustering_choices[0]:
    clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
    clustering_model.fit(X=embedding)
    clusters = clustering_model.labels_
else:
    clusters = community_detection(embeddings=embedding, min_community_size=25, threshold=0.75)

cluster = [0] * len(review_df)
for index, current in enumerate(clusters):
    for row in current:
        cluster[row] = index + 1

print('cluster time: {}'.format(now() - cluster_start))

cluster time: 0:00:00.117511


In [10]:
from math import sqrt
import pandas as pd
from plotly.express import scatter
size = int(sqrt(len(sentences)))
sentence_df = pd.DataFrame(data={'text': sentences, 'rating': review_df['review-label'].values.tolist(),
                                 'cluster': cluster}).sort_values(by=['cluster', 'rating']).reset_index()
sentence_df[['row', 'column']] = pd.DataFrame(data={'row': [row // size for row in range(len(sentences))],
                                                    'column': [column % size for column in range(len(sentences))],})
sentence_df['cluster'].value_counts()

cluster
0    288
1    100
2     62
3     53
4     26
Name: count, dtype: int64

In [11]:
scatter(data_frame=sentence_df, 
#         pd.DataFrame(data=
#                                  {'text': sentences, 
#                                        'row': [row // size for row in range(len(sentences))],
#                          'column': [column % size for column in range(len(sentences))],
#                          'cluster': cluster}),
        x='row', y='column', color='rating',
        facet_col='cluster', facet_row='rating',
        color_continuous_scale='dense',
        hover_name='text',
        height=900,
       )

In [12]:
colors = ['aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
             'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
             'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
             'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
             'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
             'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
             'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
             'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl',
             'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn',
             'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu',
             'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar',
             'spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn',
             'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd']